In [15]:
import random
import numpy as np

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, LSTM

Using TensorFlow backend.


### Define an LSTM Network 

In [13]:
class LSTM_function():
    
    def __init__(self, inputs=4, outputs=1, steps=10):
        self.inputs, self.outputs = inputs, outputs
        self.learning_rate = 0.001        

        self.model = Sequential(name="LSTM_network-in:%d-out:%d-steps:%d"%(inputs,outputs,steps))
        
        self.model.add(LSTM(steps, activation='relu', input_shape=(steps, inputs), return_sequences=False, stateful=False))
        self.model.add(Dense(16, activation="relu"))
        self.model.add(Dense(outputs, activation='linear'))
        
        self.model.compile(loss="mse", optimizer=Adam(lr=self.learning_rate))
        self.model.summary()

    def predict(self, s, a):
        s_batch = np.reshape(s, [1]+list(s.shape))
        return self.model.predict(s_batch)[0][a]

    def update(self, s, a, y):
        s_batch = np.reshape(s, [1]+list(s.shape))   # calling predict was causing
        q_values = self.model.predict(s_batch)[0]       # bad inheritance behavior
        q_values[a] = y
        q_values_batch = np.reshape(q_values, [1, self.outputs])
        self.model.fit(s_batch, q_values_batch, verbose=0)

    def update_batch(self, states, targets):
        self.model.train_on_batch(states, targets)

### Train an LSTM Network to predict the sin function
$f([sin(x_{t-4}+dx),...,sin(x_{t-1}+dx)])=sin(x_t)$

In [14]:
def gen_input(nsteps):
    x_ini = np.random.rand()
    x_step = np.random.rand()
    sin_seq = np.array([ np.array([np.sin(x_ini+i*x_step)]) for i in range(nsteps+1)]) 
    y = sin_seq[-1][0]
    s = sin_seq[:-1]
    return s,y

nsteps = 10
f = NN_function(inputs=nsteps, outputs=1)
f_lstm = LSTM_function(inputs=1, outputs=1, steps=nsteps)

n = 10000
for _ in range(n):            # train the network with input-output pairs
    s,y = gen_input(nsteps)
    f.update(s,0,y)
    f_lstm.update(s,0,y)

error, error_lstm = 0, 0
for _ in range(int(n/10)):         # test the network with mean squared error (MSE)
    s,y = gen_input(nsteps)    
    error += np.power(f.predict(s,0)-y,2)  # MSE
    error_lstm += np.power(f_lstm.predict(s,0)-y,2)  # MSE
    
print("MSE errors (NN,LSTM) =", error, error_lstm)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "NN-in:10-out:1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_2 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 249
Trainable params: 249
Non-trainable params: 0
_________________________________________________________________
Model: "LSTM_network-in:1-out:1-steps:10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10)                480       
__________________________